# Cross Validation

1. Reservar 1/k de los datos como muestra reservada.
2. Entrenar el modelo con los datos restantes
3. Aplicar (puntuar) el modelo a la retención de 1/k y registrar las métricas de evaluación del modelo.
4. Restaurar el primer 1/k de los datos y reservar el siguiente 1/k (excluyendo los registros que se seleccionaron la primera vez)
5. Repetir los pasos 2 y 3
6. Repetir hasta que cada registro se haya utilizado en la parte reservada
7. Promediar o combinar las métricas de evaluación del modelo.

In [31]:
# Reto: Seleccionar las caracteristicas que creen que son más importantes y hacer el proceso de regresión lineal y obtener métricas e interpretar.

In [32]:
import pandas as pd

In [33]:
data = pd.read_csv("../Datasets/house_sales.csv",delimiter="\t")

In [34]:
predictors = ['SqFtLot','Bedrooms','SqFtFinBasement','Bathrooms','NewConstruction']
outcome='AdjSalePrice'

In [35]:
data[predictors]

,SqFtLot,Bedrooms,SqFtFinBasement,Bathrooms,NewConstruction
1,9373,6,0,3.00,False
2,20156,4,1452,3.75,True
3,26036,4,900,1.75,False
4,8618,5,1640,3.75,False
5,8620,4,0,1.75,False
...,...,...,...,...,...
27057,5468,3,590,1.75,False
27058,23914,4,910,4.50,False
27061,11170,4,0,1.00,False
27062,6223,3,0,2.00,False


In [36]:
data = pd.get_dummies(data[predictors],drop_first=True)
data

,SqFtLot,Bedrooms,SqFtFinBasement,Bathrooms,NewConstruction
1,9373,6,0,3.00,False
2,20156,4,1452,3.75,True
3,26036,4,900,1.75,False
4,8618,5,1640,3.75,False
5,8620,4,0,1.75,False
...,...,...,...,...,...
27057,5468,3,590,1.75,False
27058,23914,4,910,4.50,False
27061,11170,4,0,1.00,False
27062,6223,3,0,2.00,False


In [37]:
data['NewConstruction'] = [1 if d else 0 for d in data['NewConstruction']]
data

,SqFtLot,Bedrooms,SqFtFinBasement,Bathrooms,NewConstruction
1,9373,6,0,3.00,0
2,20156,4,1452,3.75,1
3,26036,4,900,1.75,0
4,8618,5,1640,3.75,0
5,8620,4,0,1.75,0
...,...,...,...,...,...
27057,5468,3,590,1.75,0
27058,23914,4,910,4.50,0
27061,11170,4,0,1.00,0
27062,6223,3,0,2.00,0


In [38]:
data = data[data['SqFtFinBasement']>0]
data

,SqFtLot,Bedrooms,SqFtFinBasement,Bathrooms,NewConstruction
2,20156,4,1452,3.75,1
3,26036,4,900,1.75,0
4,8618,5,1640,3.75,0
8,14659,4,620,1.75,0
12,5200,3,600,1.00,0
...,...,...,...,...,...
27053,5019,4,790,1.50,0
27054,6021,4,640,3.25,0
27055,5150,2,800,1.00,0
27057,5468,3,590,1.75,0


In [37]:
# Entrenamiento con los datos